# 准备方法

In [1]:
import jieba
from collections import Counter
from pathlib import Path
from tqdm import tqdm
import os
import sys

sys.path.append('../')

from examination_paper_classifier import extract_text
from examination_paper_classifier import is_all_chinese_or_english
from examination_paper_classifier import classify_paper_type

def most_common_words(text, n=40):
    words = jieba.cut(text)
    words = filter(lambda word: len(word) > 1 and is_all_chinese_or_english(word), words)
    word_count = Counter(words)
    most_common = word_count.most_common(n)
    most_common_dict = dict(most_common)
    return most_common_dict


def add_dicts(dicts_list):
    result_dict = {}
    for d in dicts_list:
        for key, value in d.items():
            if key in result_dict:
                result_dict[key] += value
            else:
                result_dict[key] = value
            
    return sorted(result_dict.items(), key=lambda x: x[1], reverse=True)


def search_most_keywords_by_file(file_path):
    file_path = Path(file_path)
    try:
        text = extract_text(str(file_path), file_path.suffix)
        return most_common_words(text)
    except:
        return {}
    
    
def search_most_keywords_by_dir(dir_path, file_name_keyword=[]):
    dicts_list = []
    for file_path in tqdm(list(Path(dir_path).rglob('*.doc*'))):
        file_name = file_path.name
        if not file_name_keyword:
            dicts_list.append(search_most_keywords_by_file(file_path))
        elif any([word in file_name for word in file_name_keyword]):
            dicts_list.append(search_most_keywords_by_file(file_path))
            
    return add_dicts(dicts_list)

# 首先查看各种类型试卷所需要的关键词

In [127]:
all_categorys_total = {"化学":[], "历史":[], "地理":[], "政治":[], "数学":[], "物理":[], "生物":[], "语文":[] }


for file_path in tqdm(list(Path("../data/docx/题库").rglob('*.doc*'))):
    file_name = file_path.name
    
    for category in all_categorys_total:
        if category in file_name:
            all_categorys_total[category].append(search_most_keywords_by_file(file_path))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 8285/8285 [01:56<00:00, 70.99it/s]


In [128]:
for category in all_categorys_total:
    keywords_total = add_dicts(all_categorys_total[category])[:20]
    print(category)
    print(keywords_total)
    print()

化学
[('反应', 12393), ('pic', 11694), ('溶液', 8524), ('生成', 4338), ('正确', 3100), ('下列', 3069), ('mol', 3069), ('元素', 2812), ('物质', 2766), ('发生', 2744), ('离子', 2621), ('答案', 2377), ('方程式', 2218), ('OH', 1959), ('分析', 1857), ('氧化', 1826), ('结构', 1776), ('实验', 1768), ('加入', 1616), ('化学', 1612)]

历史
[('材料', 2324), ('中国', 1491), ('答案', 968), ('历史', 893), ('发展', 887), ('根据', 810), ('所学', 802), ('结合', 799), ('经济', 735), ('知识', 718), ('国家', 674), ('社会', 493), ('世纪', 486), ('世界', 466), ('美国', 464), ('政治', 440), ('英国', 331), ('主要', 325), ('要求', 322), ('问题', 316)]

地理
[('pic', 977), ('主要', 948), ('地区', 826), ('发展', 721), ('我国', 657), ('城市', 598), ('影响', 583), ('回答', 480), ('原因', 425), ('形成', 405), ('分布', 389), ('答案', 370), ('完成', 358), ('问题', 348), ('变化', 343), ('经济', 325), ('人口', 305), ('材料', 283), ('分析', 272), ('错误', 251)]

政治
[('发展', 2362), ('文化', 1683), ('材料', 1354), ('中国', 1339), ('经济', 1314), ('社会', 1094), ('我国', 1090), ('国家', 994), ('企业', 761), ('问题', 659), ('人民', 658), ('政府', 626), ('答案', 600

# 读取测试数据看是否存在偏差(运行完分类脚本后)

In [332]:
import pandas as pd

all_file_df = pd.read_csv("../classifier.csv")

all_file_df['probability'] = pd.to_numeric(all_file_df['probability'])

threshold = 0.5

positive_df = all_file_df[all_file_df['probability'] > threshold]
negative_df = all_file_df[all_file_df['probability'] <= threshold]

all_file_df

,file_path,target_path,probability,type
0,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,1.211337e-24,NaN
1,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,6.115465e-26,NaN
2,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,1.424632e-14,NaN
3,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,6.168316e-12,NaN
4,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,./data/docx/18 A股脱水研报/A股脱水研报（2018年10月）/【公益知识库z...,3.395665e-19,NaN
...,...,...,...,...
59574,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,9.999979e-01,indefinable
59575,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,9.993577e-01,indefinable
59576,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,9.973776e-01,indefinable
59577,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,./data/docx/202306/2023全网考研资料免费分享/考研资料小合集考研资料小...,9.999971e-01,indefinable


In [333]:
len(positive_df)

10297

In [334]:
positive_df['type'].value_counts()

type
数学             3102
indefinable    1308
医学             1172
公务员            1115
政治              669
化学              605
物理              546
地理              538
语文              439
历史              411
生物              254
理综               64
文综               63
心理                5
Name: count, dtype: int64

# 通过文件路径以及文件名验证数据准确性

In [347]:
class_name = "政治"

print(len(positive_df[(positive_df["type"] == class_name) & ~positive_df["file_path"].str.contains(class_name)]))

for _, row in positive_df.iterrows():
    if row['type'] == class_name and class_name not in row['file_path']:
        print(f"{row['type']} {row['file_path']}")
        print()

410
政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年2月份更新】4900篇各类材料合集/2022年吉林省通化市第三季度梅河新区招聘人才考试精选题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年2月份更新】4900篇各类材料合集/2022年吉林松原市秋季基层治理专干招聘考试《通用知识》精选题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年2月份更新】4900篇各类材料合集/2022年辽宁云南第四季度事业单位联考ABC类全部《职测》试题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年2月份更新】4900篇各类材料合集/2022年四川成都第三季度事业单位考试《公共基础知识》精选题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年2月份更新】4900篇各类材料合集/2022年辽宁大连市秋季事业单位《职业能力倾向测验》试题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年1月份更新】党风廉政/党风廉政测试题/《中国共产党纪律处分条例》测试题【官方抖音号：资深秘书】.docx

政治 ./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年5月份更新】各类汇编/2023年5月份/每

# 在本地文件夹内搜索关键词的数量

In [11]:
search_most_keywords_by_dir("../data/docx/202307/中小学资源(1)/【中小学课程】/高中课程/北京四中高中历史精品全套/四中高三/", {})

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 49.60it/s]


[('中国', 133),
 ('发展', 130),
 ('经济', 97),
 ('资本主义', 81),
 ('政治', 74),
 ('国家', 68),
 ('革命', 66),
 ('时期', 58),
 ('思想', 58),
 ('民主', 58),
 ('英国', 54),
 ('世界', 52),
 ('世纪', 51),
 ('社会', 51),
 ('资产阶级', 51),
 ('建立', 49),
 ('制度', 45),
 ('形成', 45),
 ('日本', 42),
 ('法国', 40),
 ('民族', 38),
 ('美国', 37),
 ('战争', 33),
 ('社会主义', 32),
 ('文化', 32),
 ('人民', 32),
 ('改革', 31),
 ('统治', 30),
 ('工业革命', 30),
 ('宪法', 27),
 ('历史', 26),
 ('工业', 26),
 ('近代', 24),
 ('雅典', 23),
 ('西方', 23),
 ('年代', 22),
 ('出现', 22),
 ('成立', 22),
 ('运动', 22),
 ('意义', 21),
 ('公民', 21),
 ('地位', 21),
 ('西欧', 21),
 ('宗教改革', 21),
 ('内容', 20),
 ('格局', 20),
 ('影响', 20),
 ('帝国', 20),
 ('拿破仑', 20),
 ('欧洲', 18),
 ('共和国', 18),
 ('史观', 17),
 ('皇帝', 17),
 ('议会', 17),
 ('农民', 17),
 ('建设', 16),
 ('生产', 16),
 ('教会', 16),
 ('俄国', 16),
 ('土地', 16),
 ('冷战', 15),
 ('封建', 15),
 ('进入', 15),
 ('提出', 14),
 ('实行', 14),
 ('标志', 14),
 ('反对', 14),
 ('辛亥革命', 14),
 ('德意志', 14),
 ('获得', 14),
 ('时间', 13),
 ('主要', 13),
 ('德国', 13),
 ('两极', 13),
 ('美苏', 13),
 ('性质', 

In [8]:
search_most_keywords_by_file("../data/docx/202307/系统分析师(高级)/系统分析师参考书籍/系统分析师历年试题及答案/系统分析师历年试题及答案/2006年下半年系统分析师试卷及答案和分析(24页).doc")

{'试题': 79,
 '系统': 61,
 '问题': 53,
 '分析': 43,
 '应用': 35,
 '进行': 34,
 '技术': 31,
 '企业': 28,
 '可以': 26,
 '采用': 26,
 '信息': 26,
 '网络': 25,
 '需求': 24,
 '软件': 23,
 'SOA': 23,
 '过程': 22,
 '计算机': 22,
 '工作': 21,
 '模型': 21,
 '关于': 20,
 '项目': 20,
 '要求': 20,
 '方法': 19,
 '开发': 19,
 '一个': 18,
 'pic': 17,
 '关系': 17,
 '论述': 16,
 '考生': 16,
 '根据': 15,
 '实现': 15,
 '数据': 15,
 '表示': 15,
 '环境': 15,
 '贷款': 15,
 '任务': 14,
 '有关': 14,
 '以下': 13,
 '描述': 13,
 '风险': 13}

In [13]:
file_path = "../data/docx/202307/中小学资源(1)/【中小学课程】/高中课程/北京四中高中历史精品全套/四中高一历史/2/06.doc"

classify_paper_type(extract_text(file_path, "."+file_path.split(".")[-1]), file_name=file_path.split("/")[-1], debug=True)

[('政治', 107), ('历史', 81), ('公务员', 55), ('地理', 14), ('医学', 0), ('化学', 0), ('数学', 0), ('物理', 0), ('生物', 0), ('语文', 0), ('心理', 0), ('理综', 0), ('文综', 0), ('other', 0)]


'政治'